# Exercise 2.1.1

Build a `Pipeline` version of the `PolynomialRegressor` and search for the optimal `degree` of the polynomial.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

## Solution add PolynomialFeatures before LinearRegression in the pipeline
est = Pipeline(steps=[('poly', PolynomialFeatures()), ('lr', LinearRegression())])

## Use nested hyper-parameter naming to search over degree.
gs = GridSearchCV(estimator=est, param_grid={'poly__degree': range(1, 20)}, 
                  scoring='neg_mean_squared_error', cv=3)

gs.fit(X[:, np.newaxis], y)

fig = plot_data(X, y, fn=true_fn)
plot_estimator(gs.best_estimator_, fig)

# Exercise 2.1.2

Implement the following pipeline:

<img src="img/exercise_2_1_2.png">

And tune hyper-parameters:
  * `RandomForestRegressor(criterion='mse')` with `['mse', 'mae']`

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import ShuffleSplit


numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99999)),
    ])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('fx_selection', SelectFromModel(ElasticNet(l1_ratio=1.0), max_features=100, threshold=None))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features),
    ])

# Append estimator to preprocessing pipeline.
# Now we have a full prediction pipeline.
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', RandomForestRegressor(n_estimators=20, n_jobs=-1))])

# Use a simple train-test split for GridSearch
_, test_split = next(iter(ShuffleSplit(test_size=0.2, random_state=0).split(X_train)))
ps = PredefinedSplit(test_split)
gs = GridSearchCV(estimator=est, param_grid={'rf__criterion': ['mse', 'mae']}, 
                  scoring='neg_mean_absolute_error', cv=2, refit=True, verbose=10)

gs.fit(X_train, y_train)

print("best params: {}".format(gs.best_params_))
print("model score: %.3f" % mean_absolute_error(y_test, gs.best_estimator_.predict(X_test)))